In [1]:
!git clone https://github.com/sameerkhan7/Robot_Tool_Segmentation.git
%cd Robot_Tool_Segmentation


fatal: destination path 'Robot_Tool_Segmentation' already exists and is not an empty directory.
/content/Robot_Tool_Segmentation


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pwd

/content/Robot_Tool_Segmentation


# Downloading the data ONCE

In [4]:
# # Downloading
# !cp /content/drive/MyDrive/data.zip /content/Robot_Tool_Segmentation/


In [5]:
%cd /content/Robot_Tool_Segmentation/
# !unzip data.zip -d data

/content/Robot_Tool_Segmentation


In [6]:
#%cd /content/Robot_Tool_Segmentation
# Remove the 'data' folder and its contents
# !rm -rf data


In [7]:
# Libraries
!ls
!pwd

BAAnet.ipynb  data		EncoderModule.py      model.pth    TrainingUtils.py
BAAnet.py     data.zip		EndovisDataloader.py  __pycache__
BBAModule.py  DecoderModule.py	LICENSE		      README.md
/content/Robot_Tool_Segmentation


In [8]:
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import EndovisDataloader
import BBAModule
import EncoderModule
import TrainingUtils
import BAAnet
import DecoderModule


In [9]:
# set seed for reprodicibility
def set_seed(seed):
    random.seed(seed)  # Python random seed
    np.random.seed(seed)  # NumPy random seed
    torch.manual_seed(seed)  # PyTorch random seed for CPU
    torch.cuda.manual_seed(seed)  # PyTorch random seed for GPU
    torch.cuda.manual_seed_all(seed)  # PyTorch random seed for all GPUs
    # Ensure deterministic behavior in CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
%cd /content/Robot_Tool_Segmentation/data/data
!pwd


/content/Robot_Tool_Segmentation/data/data
/content/Robot_Tool_Segmentation/data/data


In [11]:
import os

In [12]:
os.getcwd()

'/content/Robot_Tool_Segmentation/data/data'

In [13]:
%cd ../..

/content/Robot_Tool_Segmentation


In [18]:
# get dataloaders and display test images
seed = 42
batch_size = 32
reduce_factor = 4 # how much to reduce image size by before training
num_workers = 0 # 0 for notebooks
train_dataloader, val_dataloader, test_dataloader = EndovisDataloader.getDataloaders(batch_size, reduce_factor, num_workers,seed)


['/content/Robot_Tool_Segmentation/data/data/seq_1/frames', '/content/Robot_Tool_Segmentation/data/data/seq_10/frames', '/content/Robot_Tool_Segmentation/data/data/seq_11/frames', '/content/Robot_Tool_Segmentation/data/data/seq_12/frames', '/content/Robot_Tool_Segmentation/data/data/seq_13/frames', '/content/Robot_Tool_Segmentation/data/data/seq_14/frames', '/content/Robot_Tool_Segmentation/data/data/seq_15/frames', '/content/Robot_Tool_Segmentation/data/data/seq_16/frames', '/content/Robot_Tool_Segmentation/data/data/seq_2/frames', '/content/Robot_Tool_Segmentation/data/data/seq_3/frames', '/content/Robot_Tool_Segmentation/data/data/seq_4/frames', '/content/Robot_Tool_Segmentation/data/data/seq_5/frames', '/content/Robot_Tool_Segmentation/data/data/seq_6/frames', '/content/Robot_Tool_Segmentation/data/data/seq_7/frames', '/content/Robot_Tool_Segmentation/data/data/seq_9/frames', '/content/Robot_Tool_Segmentation/data/data/test_seq_1/frames', '/content/Robot_Tool_Segmentation/data/data

## Setting Hyperparameters

In [20]:
# model and hyperparameters
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
epochs = 50
lr = 1e-4
set_seed(seed) # set random seed
#model = BAAnet.BAANet_BBA_Only().to(device)
model = BAAnet.BAANet().to(device)
#model = BAAnet.BAANet_BAF_Only().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Adam optimizer
criterion = TrainingUtils.DICELoss # loss function
#criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5) # learning rate scheduler

cuda:0


## Training

In [21]:
# train model. Output is [epoch, training_loss, mdice, miou, mdice_robot, miou_robot]
training_outputs = []

for i in range(epochs):
    training_output = TrainingUtils.train(train_dataloader, val_dataloader, model, criterion, optimizer, scheduler, i, device)
    training_outputs.append(training_output)

52it [02:42,  3.12s/it]
5it [03:26, 41.26s/it]


KeyboardInterrupt: 